# Тестовое задание

**Общие требования:**
1. приложить файл requirements.txt с пакетами, которые необходимо установить для воспроизведения результата.
1. комментировать ход решения.
1. **дополнительное задание на подумать:** есть ли метрики, которые лучше подходят для задачи?
1. **дополнительное задание на подумать:** как изменилось бы решение, если бы модель необходимо было встроить в другую систему, используя Java?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as hierarchy

from sklearn.cluster import KMeans

from sklearn.metrics import accuracy_score, classification_report, roc_curve,roc_auc_score,silhouette_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
%matplotlib inline

In [2]:
## 1. Классификация
task1data = pd.read_csv('./task1_train_v3.csv', sep=';',header=0,index_col=False)
task1data.head()

FileNotFoundError: [Errno 2] File ./task1_train_v3.csv does not exist: './task1_train_v3.csv'

**Задача:** для заданного набора данных (task1_train.csv) построить классификатор. Метрика: AUC ROC

In [ ]:
task1data.shape

В характеристике stat_2 есть пробелы (nan), лучше посмотреть как обстоят дела со всеми характеристиками

In [ ]:
task1data.isna().sum()


В среднем, у всех статистик есть такие пробелы, они небольшие и какая-то отдельная характеристика не выделяется, а значит, мы просто отфильтруем эти "плохие" наблюдения.

In [ ]:
task1data.groupby(['y']).size()

Отклик **y** распределён неплохо, ни один из 2х классов не доминирует. 

Поскольку характеристик всего 8, можно спокойно посмотреть на их гистограммы.

In [ ]:
task1data['stat_1'].hist(bins=100);

In [ ]:
task1data['stat_2'].hist(bins=100);

In [ ]:
task1data['stat_3'].hist(bins=100);

In [ ]:
task1data['stat_4'].hist(bins=100);

In [ ]:
task1data['stat_5'].hist(bins=100);

In [ ]:
task1data['stat_6'].hist(bins=100);

In [ ]:
task1data['stat_7'].hist(bins=100);

In [ ]:
task1data['stat_8'].hist(bins=100);

Судя по гистограммам, есть явные выбросы, которые стоит убрать.

In [ ]:
valid_task1data = task1data.drop(task1data[(task1data['stat_1']) > 350].index)
valid_task1data = valid_task1data.drop(valid_task1data[(valid_task1data['stat_3']  > 40)].index)
valid_task1data = valid_task1data.drop(valid_task1data[(valid_task1data['stat_4']  > 80)].index)
valid_task1data = valid_task1data.drop(valid_task1data[(valid_task1data['stat_5']  > 50)].index)
valid_task1data = valid_task1data.drop(valid_task1data[(valid_task1data['stat_6']  > 70)].index)
valid_task1data = valid_task1data.drop(valid_task1data[(valid_task1data['stat_7']  > 50)].index)
valid_task1data = valid_task1data.drop(valid_task1data[(valid_task1data['stat_8']  > 50)].index)
valid_task1data = valid_task1data.dropna()
valid_task1data.shape


In [ ]:
valid_task1data.describe()

In [ ]:

colors={'1':'green', '0':'red'}
valid_task1data['y'] = valid_task1data['y'].astype(str)
scatter_matrix(valid_task1data,figsize=(26,26), diagonal='kde',alpha=0.2, c=valid_task1data['y'].replace(colors))

In [ ]:
Диаграммы рассеивания могли нам указать на явно значимиые характеристики, но не в этом случае. После того, как убрали выбросы ещё раз взглянем на гистограмму, к примеру, первой характеристики.

In [ ]:
valid_task1data['stat_1'].hist(bins=100);

Если внимательно взглянуть на диаграммы рассеивания, то можно заметить что stat_5 и stat_6 практически линейно друг от друга зависят, проверить это можем с помощью корреляции Пирсона.

In [ ]:
valid_task1data['stat_5'].corr(valid_task1data['stat_6']),valid_task1data['stat_6'].corr(valid_task1data['stat_5'])


Что и требовалось доказать. Уберём, допустим, 5ю характеристику, она избыточна.


7 и 8 характеристики тоже смущают.


In [ ]:
valid_task1data['stat_7'].corr(valid_task1data['stat_8']),valid_task1data['stat_8'].corr(valid_task1data['stat_7'])


7ю характеристику в сад, но смущают 8я и 4я....

In [ ]:
valid_task1data['stat_8'].corr(valid_task1data['stat_4']),valid_task1data['stat_4'].corr(valid_task1data['stat_8'])


4ю тоже в сад.

In [ ]:
filtered_valid_task1data=valid_task1data.drop('stat_5',axis=1)
filtered_valid_task1data=filtered_valid_task1data.drop('stat_4',axis=1)
filtered_valid_task1data=filtered_valid_task1data.drop('stat_7',axis=1)
x = filtered_valid_task1data.drop('y',axis=1)
y = filtered_valid_task1data['y']
colors={'1':'green', '0':'red'}
filtered_valid_task1data['y'] = filtered_valid_task1data['y'].astype(str)
scatter_matrix(filtered_valid_task1data,figsize=(26,26), diagonal='kde',alpha=0.2, c=filtered_valid_task1data['y'].replace(colors))



In [ ]:
train_x,test_and_validate_x,train_y,test_and_validate_y= train_test_split(x, y, stratify=y,  test_size=0.4, random_state=42)
{'train': train_y, 'test_and_ validate': test_and_validate_y}
train_y.groupby(train_y).count()

Распилить по индексу 3000 оказалось вполне пригодным решением, **y** сохранил репрезентативность и в подвыборках.
Но мы воспользуемся **train_test_split**. Судя по тому,что значения отклик **y** принимает только 0 и 1, мы можем рассматривать эту шкалу как номинативную (бинарный индикатор), а значит можно попробовать использовать регрессию или случайный лес, а лучше GBM.

In [ ]:
validate_x,test_x,validate_y,test_y= train_test_split(test_and_validate_x, test_and_validate_y, stratify=test_and_validate_y,  test_size=0.5, random_state=42)
{'train': train_y, 'test': test_y}
validate_y.groupby(validate_y).count(),test_y.groupby(test_y).count()


Выборка валидации использована для подгонки стандартных внешних параметров модели. Сделаем что-то наподобии gridsearch только своими руками через циклы! В качестве промежуточной оценки для валидации возьму **accuracy**

In [ ]:
max_score = -100000
for criterion in ('friedman_mse','mae','mse'):
    for n_estimators in (100,110,120,130):
        for max_depth in range(1,5):
            model = GradientBoostingClassifier(random_state=52,
                                               max_depth=max_depth, # глубина каждого дерева (значение по умолчанию)
                                               n_estimators=n_estimators, # количество деревьев (значение по умолчанию)
                                               criterion=criterion # функция измерения качества разбиения
                                              )
            model.fit(train_x, train_y)
            pred_validate_y = model.predict(validate_x)
            score = accuracy_score(pred_validate_y , validate_y)            
            max_score = max(score, max_score)
            print('criterion - ' + criterion + ' n_estimators - '+ str(n_estimators)+' maxdepth - '+str(max_depth)+' : '+ str(score))
max_score

Видим, что максимальная точность достигается при параметрах 120, 4, friedman_mse. Глубину больше 4х не будем рассматривать, так как есть эвристики, что слишком глубокие деревья любят переобучаться.

In [ ]:
model = GradientBoostingClassifier(random_state=52,
                                               max_depth=4, # глубина каждого дерева (значение по умолчанию)
                                               n_estimators=120, # количество деревьев (значение по умолчанию)
                                               criterion='friedman_mse' # функция измерения качества разбиения
                                              )
model.fit(train_x, train_y)

In [ ]:
pred_test_y = model.predict_proba(test_x)
print(pred_test_y)

In [ ]:
plt.figure(figsize=(10, 8))
fpr, tpr, thresholds = roc_curve(test_y.astype('int'), pred_test_y[:,1], pos_label=1)
lw = 1
plt.plot(fpr, tpr, lw=lw, label='ROC curve ')
plt.plot([0, 1],[0, 1])
plt.xlim([0.0, 1.])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.savefig("ROC.png")
plt.show()
thresholds

Порог стоит выбрать таким, чтобы tpr - fpr был максимальным.

In [ ]:
max_index = np.argmax(tpr-fpr)
roc_score = roc_auc_score(test_y.astype('int'), pred_test_y[:,1])
print('порог - '+ str(thresholds[max_index])) 
print('true positive rate(recall) - '+ str(tpr[max_index]))# 
print('false positive rate - '+ str(fpr[max_index]))
print('roc score - ' +str(roc_score))

> как изменилось бы решение, если бы модель необходимо было встроить в другую систему, используя Java?

Первое, что приходит на ум, это организовать отдельный сервис **А** с уже предобученной моделью и производящей результат предсказания.
1. Flask в качестве библиотеки http для оргазинации сервера и любая, к примеру httpclientlite, библиотека на стороне Java для организации клиента. (если нужен real-time)
2. Организовать 2 очереди с помощью любого подходящего хранилища (mysql или тот же zookeeper), в первой будут запросы на предсказание, во второй результаты.
3. Возможно, стоит подумать об отказе от Python в пользу Hadoop кластера и SparkML на его основе (кстати опыт в этих вещах у меня есть :-) ).

В любом случае, дообучение и обновление можно организовать в виде jenkins/teamcity или другого варианта cron-таски, а саму модель хранить в виде ресурса, к примеру sklearn.externals.joblib дампа, который будет читаться сервисом **А** на старте.

> есть ли метрики, которые лучше подходят для задачи?

Основным критерием отказа от **accuracy** обычно считается несбалансированность в классах, 
у нас же объектов одного и другого типа примерно одинаковое количество, 
поэтому отказываться от accuracy не имеет смысла.

Точно так же и precision/recall/f-score не стоит списывать со счетов.

In [ ]:
def compare_to_threshold(x):
    return x > thresholds[max_index]
y_pred_test_no_prob = compare_to_threshold(pred_test_y[:,1])
y_pred_test_no_prob = y_pred_test_no_prob.astype(int)
y_pred_test_no_prob = y_pred_test_no_prob.astype(str).astype(object)

In [ ]:
accuracy_score(test_y, y_pred_test_no_prob)

In [ ]:

print(classification_report(test_y, y_pred_test_no_prob))

Задача выглядит так, что её можно проверить с помощью логистической регрессии.

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42,solver='liblinear', multi_class='ovr', penalty='l2')
# help(LogisticRegression)
model.fit(train_x, train_y)
linear_pred_test_y=model.predict_proba(test_x)
plt.figure(figsize=(10, 8))
fpr, tpr, thresholds = roc_curve(test_y.astype('int'), linear_pred_test_y[:,1], pos_label=1)
lw = 1
plt.plot(fpr, tpr, lw=lw, label='ROC curve ')
plt.plot([0, 1],[0, 1])
plt.xlim([0.0, 1.])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.savefig("ROC.png")
plt.show()
thresholds

In [ ]:
max_index = np.argmax(tpr-fpr)
roc_score = roc_auc_score(test_y.astype('int'), linear_pred_test_y[:,1])
print('порог - '+ str(thresholds[max_index])) 
print('true positive rate(recall) - '+ str(tpr[max_index]))# 
print('false positive rate - '+ str(fpr[max_index]))
print('roc score - ' +str(roc_score))

**TPR** тут конечно меньше, но и **FPR** меньше, так что тут нужно больше данных от заказчика о, том что хочется увидеть в итоге. Можно сказать, что логрегрессия относит к классу 1 большее число объектов чем GBM. В тоже время, площадь под кривой у GBM явно больше, поэтому он предпочтительнее.

## 2. Кластеризация

**Задача:**
кластеризовать заданный набор данных (task2.csv). Предложить метрику и проверить качество.

In [ ]:
task2data = pd.read_csv('./task2_v3.csv', sep=';',header=0,index_col=False)
task2data.head()

In [ ]:
task2data.describe()

Стандартизировать характеристики не нужно, так как нет какой-то доминирующей среди всех. Поэтому кстати можно спокойно использовать euclidian расстояние.  Можно начать с иерархического кластерного анализа.

Расстояние между кластерами Ward и между объектами euclidean является стандартном де-факто. 

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
linkage = hierarchy.linkage(task2data, 
                            'ward', #расстояние между кластерами
                            'euclidean' #расстояние между единичными объектами
                           )
linkage


In [ ]:
dendrogram = hierarchy.dendrogram(linkage, orientation='right')

In [ ]:
for dist in (10,15,30,60,120,190,260,400,500):
    cluster_res = hierarchy.fcluster(linkage,dist, 'distance')
    print('силуэт - ' + str(silhouette_score(task2data, cluster_res)) + ' количество кластеров - ' + str(cluster_res.max()))
    

Оценка силуэта говорит, что лучше остановиться в расстоянии между 60 и 120.

In [ ]:

task2data['hierarchy_cluster']= hierarchy.fcluster(linkage,120, 'distance')


In [ ]:
task2data.groupby('hierarchy_cluster').mean()


In [ ]:
hierarchy_cluster_result = task2data['hierarchy_cluster']
task2data.groupby('hierarchy_cluster').size()

In [ ]:
colors={'1':'green', '4':'red', '2':'blue', '3':'black', '5':'orange','6':'yellow','7':'lime','8':'violet'}
task2data['hierarchy_cluster'] = task2data['hierarchy_cluster'].astype(str)
scatter_matrix(task2data,figsize=(26,26), diagonal='kde',alpha=0.6, c=task2data['hierarchy_cluster'].replace(colors))

8 кластеров вполне достаточно. Но перепроверим с помощью метода k-средних.

In [ ]:
hierarchy_cluster_result


In [ ]:
w = {}
K = range(2,15)
for kluster_size in K:
    model = KMeans(n_clusters=kluster_size,random_state=42)
    model.fit(task2data)
    ss = silhouette_score(task2data, model.labels_)
    w[str(kluster_size)] = {'w' : model.inertia_, 'silhouette' : ss}
w

В качестве критерий качества (простите за тавтологию), использован силуэт.

In [ ]:
w_plot=list(map((lambda x: x.get('w')), w.values()))
plt.plot(K, w_plot,marker='o')


Судя по графику и цифрам, переход с 7 до 8 кластеров довольно сильно влияет на W, в отличии от дальнейших переходов. 
Остановимся так же на 8ми кластерах. Да и оценка силуэта говорит о том же.

In [ ]:
model = KMeans(n_clusters=8,random_state=42)
model.fit(task2data)
task2data['kmean_cluster'] = model.labels_

In [ ]:
task2data.groupby('kmean_cluster').mean()

Вспомним силуэт иерархической кластеризации.

In [ ]:
silhouette_score(task2data, hierarchy_cluster_result)

Получаем, что иерархическая кластеризация предпочтительнее.

К сожалению, о характеристиках каждого кластера ничего бизнесово-полезного сказать не получится, кроме простых наблюдений. В 1ом кластере характеристики 2,3,4 принимают отрицательные значения, в 5ом -
характеристики 1 и 2 положительны и т.д. 

p.s.
Можно предположить, что это температуры в населённых пунктах по четырём сезонам года, тогда становится веселее :-)